# Présentation du dataset MABe – Mouse Behavior Detection

Ce notebook se concentre uniquement sur la **compréhension et l’exploration des données brutes**
du dataset *MABe – Mouse Behavior Detection* (Kaggle).

L’objectif n’est **pas** encore de construire un modèle, mais de répondre à des questions
fondamentales sur les données :

- Quels types de fichiers composent le dataset ?
- Quelles informations contiennent les métadonnées des vidéos ?
- Comment sont stockées les trajectoires des souris ?
- À quoi correspond concrètement un fichier `.parquet` et pourquoi ce format est utilisé ?

Le but est d’acquérir une vision claire de la structure des données avant toute transformation
ou modélisation.

In [14]:
%pip install pyarrow fastparquet
%pip install reservoirpy
%pip install matplotlib
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import pandas as pd
import numpy as np
from pathlib import Path

## Chargement des métadonnées

Les fichiers `train.csv` et `test.csv` contiennent les **métadonnées associées à chaque vidéo** :
informations sur le laboratoire, les souris, les paramètres d’enregistrement et les comportements annotés.

Chaque ligne correspond à une vidéo distincte.


In [ ]:
# Chargement des métadonnées d'entraînement
train_metadata = pd.read_csv("data/train.csv")

# Aperçu des premières lignes
train_metadata.head()

,lab_id,video_id,mouse1_strain,mouse1_color,mouse1_sex,mouse1_id,mouse1_age,mouse1_condition,mouse2_strain,mouse2_color,...,pix_per_cm_approx,video_width_pix,video_height_pix,arena_width_cm,arena_height_cm,arena_shape,arena_type,body_parts_tracked,behaviors_labeled,tracking_method
0,AdaptableSnail,44566106,CD-1 (ICR),white,male,10.0,8-12 weeks,wireless device,CD-1 (ICR),white,...,16.0,1228,1068,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
1,AdaptableSnail,143861384,CD-1 (ICR),white,male,3.0,8-12 weeks,NaN,CD-1 (ICR),white,...,9.7,968,608,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
2,AdaptableSnail,209576908,CD-1 (ICR),white,male,7.0,8-12 weeks,NaN,CD-1 (ICR),white,...,16.0,1266,1100,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
3,AdaptableSnail,278643799,CD-1 (ICR),white,male,11.0,8-12 weeks,wireless device,CD-1 (ICR),white,...,16.0,1224,1100,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
4,AdaptableSnail,351967631,CD-1 (ICR),white,male,14.0,8-12 weeks,NaN,CD-1 (ICR),white,...,16.0,1204,1068,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut


## Structure des métadonnées

Les métadonnées fournissent le **contexte expérimental** de chaque vidéo :
- laboratoire d’origine
- durée et fréquence d’échantillonnage
- caractéristiques des souris
- liste des comportements annotés
- méthode de tracking utilisée

Ces informations sont essentielles pour comprendre l’hétérogénéité du dataset.


In [ ]:
TRACK_ROOT = Path("data/train_tracking")

# pick one category/folder to start
category = "AdaptableSnail"
files = list((TRACK_ROOT / category).glob("*.parquet"))

len(files), files[:3]

(17,
 [WindowsPath('data/data_raw/train_tracking/AdaptableSnail/1212811043.parquet'),
  WindowsPath('data/data_raw/train_tracking/AdaptableSnail/1260392287.parquet'),
  WindowsPath('data/data_raw/train_tracking/AdaptableSnail/1351098077.parquet')])

## Charger un fichier de tracking

Un fichier parquet stocke les détections de tracking au format long :
une ligne par (frame, mouse_id, bodypart).

In [18]:
tracking_path = files[0]
df = pd.read_parquet(tracking_path)

df.head()

,video_frame,mouse_id,bodypart,x,y
0,0,1,body_center,496.187012,376.475006
1,0,1,ear_left,494.059998,343.924011
2,0,1,ear_right,518.765015,367.362000
3,0,1,lateral_left,474.536987,370.563995
4,0,1,lateral_right,505.825012,394.937012


## Structure de base

Nous inspectons les colonnes, les types de données et les dimensions globales.

In [19]:
print("Shape:", df.shape)
print("Columns:", list(df.columns))
df.dtypes

Shape: (2677006, 5)
Columns: ['video_frame', 'mouse_id', 'bodypart', 'x', 'y']


video_frame      int32
mouse_id          int8
bodypart        object
x              float32
y              float32
dtype: object

## Statistiques clés

Nous résumons :
- parties du corps uniques
- nombre de frames
- nombre de souris
- nombre total de détections

In [20]:
bodyparts = df["bodypart"].unique()
n_frames = df["video_frame"].nunique()
n_mice = df["mouse_id"].nunique()

print("Unique bodyparts:", bodyparts)
print("Number of bodyparts:", len(bodyparts))
print("Number of frames:", n_frames)
print("Number of mice:", n_mice)
print("Total detections:", len(df))

Unique bodyparts: ['body_center' 'ear_left' 'ear_right' 'lateral_left' 'lateral_right'
 'nose' 'tail_base' 'tail_midpoint' 'tail_tip' 'neck']
Number of bodyparts: 10
Number of frames: 89966
Number of mice: 4
Total detections: 2677006


## Conclusions de l’exploration

1) Les données de tracking sont fournies au format *long* :  
   plusieurs lignes correspondent à une même `video_frame`, chaque ligne décrivant une paire *(mouse_id, bodypart)* avec ses coordonnées *(x, y)*.

2) Pour le Reservoir Computing, l’objectif est de construire une **série temporelle numérique** de la forme :

   $$
   X \in \mathbb{R}^{\text{timesteps} \times \text{features}}
   $$

   où chaque timestep correspond à une frame vidéo.

3) Le dataset contient jusqu’à **10 bodyparts par souris**, ce qui permettrait en théorie d’obtenir  
   \( D = 2 \times 10 = 20 \) features par souris si toutes les parties du corps étaient utilisées.

   Dans le pipeline d’entraînement considéré ici, **seule la bodypart `body_center` est conservée** afin de simplifier le signal d’entrée.  
   On obtient ainsi **2 features par souris (x, y)**, soit **8 features par frame** lorsque les 4 souris sont concaténées.

En pratique, les données sont restructurées à l’échelle de la frame afin de regrouper, pour chaque instant temporel, les positions des souris dans un vecteur de features exploitable par le modèle.